In [104]:
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from collections import Counter

In [4]:
input_ = open("day12.txt", "r", encoding="utf-16").read().strip().splitlines()

In [5]:
input_

['vp-BY',
 'ui-oo',
 'kk-IY',
 'ij-vp',
 'oo-start',
 'SP-ij',
 'kg-uj',
 'ij-UH',
 'SP-end',
 'oo-IY',
 'SP-kk',
 'SP-vp',
 'ui-ij',
 'UH-ui',
 'ij-IY',
 'start-ui',
 'IY-ui',
 'uj-ui',
 'kk-oo',
 'IY-start',
 'end-vp',
 'uj-UH',
 'ij-kk',
 'UH-end',
 'UH-kk']

In [6]:
demo_small = """start-A
start-b
A-c
A-b
b-d
A-end
b-end""".splitlines()
demo_small

['start-A', 'start-b', 'A-c', 'A-b', 'b-d', 'A-end', 'b-end']

In [8]:
demo_middle = """dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc""".splitlines()
demo_middle

['dc-end',
 'HN-start',
 'start-kj',
 'dc-start',
 'dc-HN',
 'LN-dc',
 'HN-end',
 'kj-sa',
 'kj-HN',
 'kj-dc']

In [78]:
demo_large="""fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW""".splitlines()
demo_large

['fs-end',
 'he-DX',
 'fs-he',
 'start-DX',
 'pj-DX',
 'end-zg',
 'zg-sl',
 'zg-pj',
 'pj-he',
 'RW-he',
 'fs-DX',
 'pj-RW',
 'zg-RW',
 'start-pj',
 'he-WI',
 'zg-he',
 'pj-fs',
 'start-RW']

In [88]:
@dataclass
class Cave:
    name: str
    is_start: bool
    is_end: bool
    is_big: bool
    connections: set[str] = field(default_factory=set)
    

In [89]:
def build_caves(input_) -> dict[str, Cave]:
    caves = {}
    for line in input_:
        for cave in line.split("-"):
            if cave not in caves:
                caves[cave] = Cave(
                    name=cave,
                    is_start=True if cave == "start" else False,
                    is_end=True if cave == "end" else False,
                    is_big=True if cave.isupper() else False,
                )
                
        left_cave, right_cave = line.split("-")
        caves[left_cave].connections.add(right_cave)
        caves[right_cave].connections.add(left_cave)
        
    return caves

In [80]:
build_caves(demo_small)

{'start': Cave(name='start', is_start=True, is_end=False, is_big=False, connections={'b', 'A'}),
 'A': Cave(name='A', is_start=False, is_end=False, is_big=True, connections={'end', 'start', 'b', 'c'}),
 'b': Cave(name='b', is_start=False, is_end=False, is_big=False, connections={'end', 'start', 'd', 'A'}),
 'c': Cave(name='c', is_start=False, is_end=False, is_big=False, connections={'A'}),
 'd': Cave(name='d', is_start=False, is_end=False, is_big=False, connections={'b'}),
 'end': Cave(name='end', is_start=False, is_end=True, is_big=False, connections={'b', 'A'})}

In [125]:
def find_all_paths(caves: dict[str, Cave]) -> list[str]:
    paths = []
    start = caves["start"]
    incomplete_paths = [[start.name, cave] for cave in start.connections]
    while incomplete_paths:
        current_path = incomplete_paths.pop()
        last_cave = caves[current_path[-1]]
        for cave in last_cave.connections:
            cave = caves[cave]
            if cave.is_start:
                continue
                
            if cave.is_end:
                paths.append([*current_path, cave.name])      
            elif cave.is_big:
                incomplete_paths.append([*current_path, cave.name])
            elif not cave.is_big:
                small_caves = Counter(c for c in current_path[1:] if not caves[c].is_big)
                small_caves_visited = 0 if not small_caves else small_caves.most_common(1)[0][1]
                if cave.name not in current_path or small_caves_visited < 2:
                    incomplete_paths.append([*current_path, cave.name])    
    
    return paths       
    

In [126]:
all_paths = find_all_paths(build_caves(demo_small))
for path in all_paths:
    print(path)
len(all_paths)

['start', 'A', 'end']
['start', 'A', 'c', 'A', 'end']
['start', 'A', 'c', 'A', 'c', 'A', 'end']
['start', 'A', 'c', 'A', 'c', 'A', 'b', 'end']
['start', 'A', 'c', 'A', 'c', 'A', 'b', 'A', 'end']
['start', 'A', 'c', 'A', 'b', 'end']
['start', 'A', 'c', 'A', 'b', 'A', 'end']
['start', 'A', 'c', 'A', 'b', 'A', 'c', 'A', 'end']
['start', 'A', 'c', 'A', 'b', 'A', 'b', 'end']
['start', 'A', 'c', 'A', 'b', 'A', 'b', 'A', 'end']
['start', 'A', 'c', 'A', 'b', 'd', 'b', 'end']
['start', 'A', 'c', 'A', 'b', 'd', 'b', 'A', 'end']
['start', 'A', 'b', 'end']
['start', 'A', 'b', 'A', 'end']
['start', 'A', 'b', 'A', 'c', 'A', 'end']
['start', 'A', 'b', 'A', 'c', 'A', 'c', 'A', 'end']
['start', 'A', 'b', 'A', 'c', 'A', 'b', 'end']
['start', 'A', 'b', 'A', 'c', 'A', 'b', 'A', 'end']
['start', 'A', 'b', 'A', 'b', 'end']
['start', 'A', 'b', 'A', 'b', 'A', 'end']
['start', 'A', 'b', 'A', 'b', 'A', 'c', 'A', 'end']
['start', 'A', 'b', 'd', 'b', 'end']
['start', 'A', 'b', 'd', 'b', 'A', 'end']
['start', 'A',

36

In [83]:
all_paths = find_all_paths(build_caves(demo_middle))
for path in all_paths:
    print(path)
len(all_paths)

['start', 'dc', 'end']
['start', 'dc', 'HN', 'end']
['start', 'dc', 'HN', 'kj', 'HN', 'end']
['start', 'dc', 'kj', 'HN', 'end']
['start', 'HN', 'end']
['start', 'HN', 'dc', 'end']
['start', 'HN', 'dc', 'HN', 'end']
['start', 'HN', 'dc', 'HN', 'kj', 'HN', 'end']
['start', 'HN', 'dc', 'kj', 'HN', 'end']
['start', 'HN', 'kj', 'HN', 'end']
['start', 'HN', 'kj', 'HN', 'dc', 'end']
['start', 'HN', 'kj', 'HN', 'dc', 'HN', 'end']
['start', 'HN', 'kj', 'dc', 'end']
['start', 'HN', 'kj', 'dc', 'HN', 'end']
['start', 'kj', 'HN', 'end']
['start', 'kj', 'HN', 'dc', 'end']
['start', 'kj', 'HN', 'dc', 'HN', 'end']
['start', 'kj', 'dc', 'end']
['start', 'kj', 'dc', 'HN', 'end']


19

In [86]:
all_paths = find_all_paths(build_caves(demo_large))
for path in all_paths:
    print(path)
len(all_paths)

['start', 'RW', 'zg', 'end']
['start', 'RW', 'zg', 'RW', 'pj', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'pj', 'RW', 'he', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'pj', 'RW', 'he', 'DX', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'pj', 'DX', 'he', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'pj', 'DX', 'he', 'DX', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'pj', 'DX', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'pj', 'he', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'pj', 'he', 'DX', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'he', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'he', 'RW', 'pj', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'he', 'RW', 'pj', 'DX', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'he', 'pj', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'he', 'pj', 'DX', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'he', 'DX', 'pj', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'he', 'DX', 'pj', 'DX', 'fs', 'end']
['start', 'RW', 'zg', 'RW', 'he', 'DX', 'fs', 'end']
['start', 'RW', 'zg', 'pj', 'fs', 'end']
['start', 

226

In [127]:
all_paths = find_all_paths(build_caves(input_))
#for path in all_paths:
#    print(path)
len(all_paths)

143562

Part Two

After reviewing the available paths, you realize you might have time to visit a single small cave twice. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named start and end can only be visited exactly once each: once you leave the start cave, you may not return to it, and once you reach the end cave, the path must end immediately.